In [1]:
from crypto_env import Market
import pandas as pd
import numpy as np
from utils import *
from ac import Actor, Critic
from dqn import ReplayMemory, DQN_Agent
import gym
import torch
import gym_anytrading

from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv

In [2]:
df = pd.read_csv('data/raw/FTX/btc_usd_1h.csv', header=None)
df.columns = ['ts', 'open', 'high', 'low', 'close', 'volume']

In [3]:
market_data = df.copy()
market_data = market_data.drop("ts", axis=1)

In [4]:
market_train = market_data[:int(len(market_data)/2)]
market_test = market_data[int(len(market_data)/2):]

### DQN on Market Environment

In [ ]:
env = Market(market_train, initial_balance=1000000)

agent = DQN_Agent(memory_length=500, state_dim=len(env.observation_space.sample()), num_actions=3)

train_dqn(agent, env, batch_size=32, discount=0.99, eps_init=1, eps_decay=0.99, eps_min=0.1, num_actions=3)


Episode:  0


In [ ]:
market_train.head(10)

### Stable-Baselines DQN on Market Environment

In [ ]:
gym.register(
    id='Market-v0',
    entry_point='crypto_env:Market'
)

env_maker = lambda: gym.make(
    'Market-v0',
    market_data = market_train,
    initial_balance = 1000000
)

env = DummyVecEnv([env_maker])

In [ ]:
agent = DQN(policy='MlpPolicy', env=env)
trained_dqn = agent.learn(total_timesteps=100000)

In [ ]:
test_env_maker = lambda: gym.make('Market-v0', market_data=market_test.reset_index().drop('index',axis=1), initial_balance=1000000)
test_env = test_env_maker()

In [ ]:
with torch.no_grad():
    obs = test_env.reset()
    
    s = obs
    while s is not None:
        a, _ = trained_dqn.predict(torch.FloatTensor(s))
        s_next, r, done, info = test_env.step(a)
        
        if done:
            s = None
        else:
            s = s_next
# test_env.close()

In [ ]:
test_env.render()

In [ ]:
test_env.render_history()

In [ ]:
actor = Actor(8)
critic = Critic(8)

In [ ]:
train_ac(actor, critic, env, 0.99)

In [ ]:
import torch.optim as optim

In [ ]:
GAMMA = 0.99

In [ ]:
a_opt = optim.Adam(actor.parameters(), lr=0.001)
c_opt = optim.Adam(critic.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [ ]:
def select_action(mu, sigma):
    m = torch.distributions.Normal(mu, sigma)
    a = m.sample()
    return a, m.log_prob(a)

In [ ]:
# s = torch.FloatTensor(env.reset())
s = s.detach()
mu, sigma = actor(s)
mu = nn.Tanh()(mu)
sigma = nn.Sigmoid()(sigma) + 1e-5

a, lp = select_action(mu, sigma)
print(a)

s_next, r, done, info = env.step(a)
s_next = torch.FloatTensor(s_next)
s_next = s_next.detach()

label = r + GAMMA * critic(s_next)
c_opt.zero_grad()
c_loss = criterion(critic(s), label)
c_loss.backward(retain_graph=True)
c_opt.step()


a_opt.zero_grad()
a_loss = (r + GAMMA * critic(s_next) - critic(s)) * -lp
a_loss.backward()
a_opt.step()

In [ ]:
env.render()

In [ ]:
market_data.head(10)

In [ ]:
env.render()

In [ ]:
# prediction is v(s)
# label is r + v(s')